In [11]:
import nltk # pip install nltk
import re # pip install re
import string
import spacy #pip install spacy
import random

from spacy.lang.pt.examples import sentences
from nltk.chat.util import Chat, reflections
from bs4 import BeautifulSoup # pip install bs4
from urllib.request import urlopen
from random import choice, randint
from sklearn.metrics.pairwise import cosine_similarity # pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer # pip install scikit-learn

# pip install lxml => erro de parser
# python -m spacy download pt_core_news_sm => erro de linguagem

In [12]:
txt_url = urlopen("https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial").read()
txt_html = BeautifulSoup(txt_url, 'lxml').find_all('p')
texto_pag_web = ''
for i in txt_html:
  texto_pag_web += i.text.lower()
lista_sentencas = nltk.sent_tokenize(texto_pag_web)

In [13]:
nlp = spacy.load("pt_core_news_sm")

In [14]:
# palavras que o modelo irá ignorar
stop_words = spacy.lang.pt.stop_words.STOP_WORDS
# pontuações que o modelo irá ignorar
stop_punct = string.punctuation

def preprocessamento(texto): #preparando o texto para ser processado pelo spacy
  # tirar urls
  texto = re.sub(r"https?://[A-Za-z0-9./]+",' ',texto)
  # tirar espaços em branco
  texto = re.sub(r" +", ' ',texto)
  # tirar radical (lematização)
  documento = nlp(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in stop_punct]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  return lista

In [15]:
# guardar as sentenças que serão pré-processadas pela função em uma lista
lista_sentencas_preprocessada = []
for i in range(len(lista_sentencas)):
  lista_sentencas_preprocessada.append(preprocessamento(lista_sentencas[i]))

In [19]:
textos_boas_vindas_entrada = ('hey', 'olá', 'opa', 'oi', 'eae')
textos_boas_vindas_respostas = ('hey', 'olá', 'opa', 'oi', 'bem-vindo', 'como você está?')
textos_saida = ('sair','tchau','exit','esc')

def responder_saudacao(texto):
  for palavra in texto.split():
    if palavra.lower() in textos_boas_vindas_entrada:
      return random.choice(textos_boas_vindas_respostas)  

In [17]:
def responder(texto_usuario):
  resposta_chatbot = ''
  lista_sentencas_preprocessada.append(texto_usuario)

  tfidf = TfidfVectorizer()
  palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessada)

  similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

  indice_sentenca = similaridade.argsort()[0][-2]
  vetor_similar = similaridade.flatten()
  vetor_similar.sort()
  vetor_encontrado = vetor_similar[-2]

  if (vetor_encontrado == 0):
    resposta_chatbot = resposta_chatbot + 'Desculpe, mas não entendi!'
    return resposta_chatbot
  else:
    resposta_chatbot = resposta_chatbot + lista_sentencas[indice_sentenca]
    return resposta_chatbot

In [20]:
print('Olá, sou um chatbot e vou responder perguntas sobre inteligência artificial: ')
while True:
  print('Usuário: ')
  texto_usuario = input().lower()
  if texto_usuario not in textos_saida:
    if responder_saudacao(texto_usuario) != None:
      print('Chatbot: ')
      print(responder_saudacao(texto_usuario))
      print()
    else:
      print('Chatbot: ')
      print(responder(preprocessamento(texto_usuario)))
      print()
      lista_sentencas_preprocessada.remove(preprocessamento(texto_usuario))
  else:
    print('Chatbot: Até breve!')
    break

Olá, sou um chatbot e vou responder perguntas sobre inteligência artificial: 
Usuário: 
Chatbot: 
opa

Usuário: 
Chatbot: 
outra definição de inteligência artificial é a inteligência que surge de um "dispositivo artificial".

Usuário: 
